# TITLE

# SCREEN CAPTURE

In [1]:
def one_time():
    import pyscreenshot as ImageGrab
    import time
    image_folder="captured_image"
    for i in range(0,100):
        time.sleep(8)
        im = ImageGrab.grab(bbox = (60,170,400,550))
        print("saved...........",i)
        im.save(image_folder +str(i)+'.png')
        print("clear screen now and redraw now........")

# GENERATE DATASET

In [2]:
import cv2
import csv
import glob

header  = ["label"]
for i in range(0,784):
    header.append("pixel"+str(i))
    
with open('dataset.csv','a') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    
for label in range(10):
    dirlist = glob.glob("Captured_image/"+str(label)+"/*.png")
    
    for img_path in dirlist:
        im = cv2.imread(img_path)
        im_gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
        im_gray = cv2.GaussianBlur(im_gray,(15,15),0)
        regionofinterest = cv2.resize(im_gray,(28,28),interpolation = cv2.INTER_AREA)
        
        data = []
        data.append(label)
        rows,cols = regionofinterest.shape
        
        for i in range(rows):
            for j in range(cols):
                k = regionofinterest[i,j]
                if k>100:
                    k=1
                else:
                    k=0
                data.append(k)
                
        with open('dataset.csv','a') as f:
            writer = csv.writer(f)
            writer.writerow(data)
                

# LOAD THE DATASET

In [3]:
import pandas as pd
from sklearn.utils import shuffle
data = pd.read_csv('dataset.csv')
data = shuffle(data)
data

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
831,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
591,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
971,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
927,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
89,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
531,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
959,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
793,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# SEPARATION OF DEPENDENT AND INDEPENDENT VARIABLE

In [4]:
X = data.drop(["label"],axis=1)
Y = data["label"]

# TRAIN-TEST SPLIT

In [5]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size = 0.2)

# FIT THE MODEL USING SVC AND ALSO TO SAVE THE MODEL USING USING JOBLIB

In [6]:
import joblib
from sklearn.svm import SVC
classifier = SVC(kernel="linear",random_state=6)
classifier.fit(x_train,y_train)
joblib.dump(classifier,"model/digit_recognizer")

['model/digit_recognizer']

# CALCULATING ACCURACY

In [7]:
from sklearn import metrics
prediction = classifier.predict(x_test)
print("Accuracy= ",metrics.accuracy_score(prediction,y_test))

Accuracy=  0.995


# PREDICTION OF IMAGE DRAWN IN PAINT

In [ ]:
import joblib
import cv2
import numpy as np
import time
import pyscreenshot as ImageGrab

model = joblib.load("model/digit_recognizer")
image_folder = "img/"

while True:
    img=ImageGrab.grab(bbox=(60,170,400,500))
    img.save(image_folder+"img.png")
    im = cv2.imread(image_folder+"img.png")
    im_gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
    im_gray = cv2.GaussianBlur(im_gray,(15,15),0)
    
    #Threshold the image
    ret,im_th = cv2.threshold(im_gray, 100, 255, cv2.THRESH_BINARY)
    regionofinterest = cv2.resize(im_th,(28,28),interpolation = cv2.INTER_AREA)
    
    rows,cols = regionofinterest.shape
    
    X = []
    for i in range(rows):
            for j in range(cols):
                k = regionofinterest[i,j]
                if k>100:
                    k=1
                else:
                    k=0
                X.append(k)
                
    predictions = model.predict([X])
    print("Prediction:",predictions[0])
    cv2.putText(im,"Prediction is:"+str(predictions[0]), (20,20), 0, 0.8, (0,255,0), 2, cv2.LINE_AA)
    
    cv2.startWindowThread()
    cv2.namedWindow("Result")
    cv2.imshow("Result",im)
    cv2.waitKey(10000)
    if cv2.waitKey(1) == 13:
        break
        
cv2.destroyAllWindows()

Prediction: 0
Prediction: 1
Prediction: 3
Prediction: 4
Prediction: 4
Prediction: 8
